**<font color = black size=6>实验四:决策树</font>**

**<font color = blue size=4>第一部分:函数介绍</font>**

In [2]:
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

1)Counter类的使用

In [2]:
x=np.array([[0,133,1],[0,132,0],[0,133,0]])
#使用Counter类对数组第2列进行遍历
counter = Counter(x[:,1])
#第2列中有1个132和2个133，输出该counter对象可以统计这列的数值情况，便于之后的统计
print(counter)
#因为第2列中没有为0的值，所以返回0
print(counter[0])
#因为第2列中有2个133，所以返回2
print(counter[133])
#一般的字典操作方法都能在该类中使用，例如可以通过values函数返回该列的非重复值的个数，方便对某列的非重复值的个数进行查看
print(counter.values())
#可以输出所有非重复值
print(list(counter))


Counter({133: 2, 132: 1})
0
2
dict_values([2, 1])
[133, 132]


2)使用numpy中的unique实现非重复值的提取

In [7]:
x=np.array([[0,133,1],[0,132,0],[0,133,0]])
a=np.unique(x[:])
a1=np.unique(x[:,1])
counter = Counter(x[:,1])
print(a,a1)
print(counter)
print(len(a))
a1
# print(math.log2(8))

[  0   1 132 133] [132 133]
Counter({133: 2, 132: 1})
4


array([132, 133])

**<font color = blue size=4>第二部分:实验任务</font>**

<span style="color:purple">该数据集(train_titanic.csv)为分类数据集，为泰坦尼克号的部分乘客信息以及最后是否生还。包括了四个属性值以及一个标记属性(即为Survived类型,代表是否生还),属性信息分别为Sex(性别)，sibsp(堂兄弟妹个数)，Parch(父母与小孩的个数)，Pclass(乘客等级)  
其中该数据集无缺失值和异常值，且所有连续变量已自动转换为离散变量,标记(Survived类型)也自动转变为离散变量0和1，所以你无需进行数据预处理，可以直接使用该数据集。</span>

In [5]:
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

<span style="color:purple">1) 使用pandas库将训练数据集'train_titanic.csv'载入到Dataframe对象中</span>

In [6]:
# train_frame = pd.read_csv('train_titanic.csv')
# x = np.array([[0],[1]])
# counter = Counter(x[:,0])
# print(counter)

<span style="color:purple">2) 编写函数，给定任何标记数组计算其信息熵  
    输入：标记数组  
    输出：该数组对应的信息熵  
    计算信息熵公式:
某数组包含K个不同的取值，样本为第k(k=1,2,...,K)个值的数量所占比例为p_k,则其信息熵为$$Ent=-\sum_{k=1}^K p_k log_2 p_k$$</span>
    
    
<span style="color:purple">例:  
    输入:[[0],[1]]   
    输出:(-$\frac{1}{2}$ $log_2$ $\frac{1}{2}$)+(-$\frac{1}{2}$ $log_2$ $\frac{1}{2}$)=1</span>

In [7]:
def entropy(label):
    label = label.reshape(len(label),1)
    counter = Counter(label[:,0])
    a=np.unique(label[:,0])
    ent=0
    m = len(label)
    for i in range(len(a)):
        ent -= counter[a[i]]/m*np.log2(counter[a[i]]/m)
    
    return ent

<span style="color:purple">3) 编写函数，函数功能为将所给的数据集按照指定维度dimension进行划分为若干个不同的数据集  
    【输入】：属性集合，标记集合，维度索引  
    【输出】：划分后所得到的子树属性集合，子树标记集合</span>

<span style="color:purple">例子:  
    【输入】:feature(属性值集合):[[0,0,0],[0,0,1],[1,0,2]]  
    label(标记集合):[[0],[1],[2]]  
    划分维度:0</span>  
    
<span style="color:purple">【输出】:[[0,0,0],[0,0,1]]和[[1,0,2]]  
    [[0],[1]]和[[2]]  
    tips:即将feature按其第0维度进行划分，由于feature的0维有0和1两个不同的数值，所以feature划分为[[0,0,0],[0,0,1]]和[[1,0,2]]，label划分为[[0],[1]]和[[2]]</span>

In [8]:
def split(feature, label, dimension):
    label = label.reshape(len(label),1)
    a=np.unique(feature[:,dimension])
#     split_feature = np.empty(len(a))
    split_feature = []
    split_label = []
    for i in range(len(a)):
        split_feature.append([])
        split_label.append([])
    for i in range(len(label)):
        for j in range(len(a)):
            if feature[i,dimension]==a[j]:
                split_feature[j].append(feature[i,:])
                split_label[j].append(label[i,:])
#         split_feature[j] = np.array(split_feature[j])
#         split_label[j] = np.array(split_label[j])
    for i in range(len(a)):
        split_feature[i] = np.array(split_feature[i])
        split_label[i] = np.array(split_label[i])
    split_feature = np.array(split_feature)
    split_label = np.array(split_label)
    return split_feature,split_label

<span style="color:purple">4) 编写函数，函数功能为进行【一次】决策树的结点划分，遍历找出该属性集合中信息增益(使用ID3算法中的公式计算)【最大】的属性  
    输入：属性集合，标记集合  
    输出：该次划分的最佳信息增益，最佳划分维度  
    计算信息增益公式:  
    某数据集D有若干属性值以及对应的标记值，其总样本大小为|D|,这里取其中一个属性类型feature,该特征包含V个不同的取值，属性值为第v(v=1,2,...,V)个值的数量为|$D^v$|$(\sum_{v=1}^VD^v=|D|)$,则该属性值对应的信息增益为为$$Gain(D,feature)=Ent(D)-\sum_{v=1}^K \frac{|D^v|}{D} Ent(D^v)$$
所以该函数中你需要计算出每个属性值的信息增益并输出，然后返回最大的信息增益并返回该特征的维数以及最大的信息增益值</span>

In [9]:
def one_split_ID3(feature, label):
    best_entropy = -100
    best_dimension = -1

    tot = entropy(label)
#     print(feature.shape)
#     print(label.shape)
    for i in range(len(feature[0])):
        tem = tot
        
        split_feature,split_label=split(np.c_[feature,label],label,i)
        for j in range(len(split_label)):
            tem-=len(split_label[j])/len(label)*entropy(split_label[j])
        if tem>best_entropy:
            best_entropy=tem
            best_dimension=i
    return best_entropy, best_dimension

<span style="color:purple">5) 编写函数，函数功能为进行【一次】决策树的结点划分，遍历找出该属性集合中信息增益率(使用C4.5算法中的公式计算)【最大】的属性  
    输入：属性集合，标记集合  
    输出：该次划分的信息增益率，最佳维度  
    计算信息增益率公式:  
    某数据集D有若干属性值以及对应的标记值，其总样本大小为|D|,这里取其中一个属性类型feature,该属性包含V个不同的取值，属性值为第v(v=1,2,...,V)个值的数量为|$D^v$|$(\sum_{v=1}^V|D^v|=|D|)$,该属性本身的信息熵为Ent(feature),则该属性值对应的信息增益率为$$Gain\_ratio(D,feature)=\frac{Gain(D,feature)}{Ent(feature)}$$
所以该函数中你需要输出每个特征的信息增益率，之后返回最大的信息增益率对应的属性维数以及最大的信息增益率</span>

In [13]:
def one_split_C4_5(feature, label):
    best_entropy = -100
    best_dimension = -1

    tot = entropy(label)
 
    for i in range(len(feature[0])):
        tem = tot
        entf = entropy(feature[:,i])
        split_feature,split_label=split(np.c_[feature,label],label,i)
        
        for j in range(len(split_label)):
            tem-=len(split_label[j])/len(label)*entropy(split_label[j])
        tem /= entf
        if tem>best_entropy:
            best_entropy=tem
            best_dimension=i
    return best_entropy, best_dimension

<span style="color:purple">6) 编写函数，进行【一次】决策树的结点划分，遍历找出该属性集合中基尼系数(使用CART算法中的公式计算)最小的属性以及最佳的划分值  
    输入：属性集合，标记集合  
    输出：该次划分的最佳的基尼系数，最佳维度，最佳划分值  
    计算基尼系数公式:  
    某数据集D有若干属性值以及对应的标记值，其总样本大小为|D|,该集合中样本标记类别总共有K类，第k类样本所占比例为$p_k$(k=1,2,..,K)则该数据集对应的基尼系数为$$Gini(D)=1-\sum_{k=1}^K{p_k}^2$$  
    而取其中一个属性类型feature，选定该类型的一个值value，根据feature这个属性是否为value将数据集分为两个子集$D_1$和$D_2$,则该属性feature对应的基尼系数为$$Gini\_index(D,feature)=\frac{|D_1|}{|D|}Gini(D_1)+\frac{|D_2|}{|D|}Gini(D_2)$$
所以该函数中你需要首先找出每列中的非重复值，之后根据该列的非重复值进行分类，计算出该列中以该值进行分类的基尼系数，计算出每个属性的每个非重复值的基尼系数，之后找到最小的基尼系数对应的属性维数以及对应的分类值</span>

In [14]:
def one_split_CART(feature, label):
    d = len(feature)
    best_entropy = 100
    best_dimension = -1
    best_value = -1
    for i in range(len(feature[0])):
        counter = Counter(feature[:,i])
        a=np.unique(feature[:,i])
        
        for k in a:
            tem10 = 0
            tem11 = 0
            tem20 = 0
            tem21 = 0
            for j in range(d):
                if feature[j,i] == k:
                    if label[j] == 0:
                        tem10 += 1
                    else:
                        tem11 += 1
                else:
                    if label[j] == 0:
                        tem20 += 1
                    else:
                        tem21 += 1
            tem10 /= counter[k]
            tem11 /= counter[k]
            tem20 /= (d-counter[k])
            tem21 /= (d-counter[k])
            gini = counter[k]/d*(1-tem10*tem10-tem11*tem11)+(d-counter[k])/d*(1-tem20*tem20-tem21*tem21)
            
            if gini < best_entropy:
                best_entropy = gini
                best_dimension = i
                best_value = k
            
    return best_entropy,best_dimension,best_value

<span style="color:purple">7) 应用之前你在第4、5、6个部分编写的三个函数，在训练数据集'train_titanic.csv'上依次使用这些函数进行【一次】结点划分，并输出对应的最佳属性维数以及相应的信息增益值/信息增益率/(基尼系数和分类值)</span>

In [15]:
train_frame = pd.read_csv('train_titanic.csv')
D = np.array(train_frame)
feature = D[:,0:D.shape[1]-1]
label = D[:,D.shape[1]-1:D.shape[1]]
print(one_split_ID3(feature, label))
print(one_split_C4_5(feature, label))
print(one_split_CART(feature, label))

(0.2318094843987734, 0)
(0.24383747685657423, 0)
(0.32380857047523715, 0, 0)


**<font color = blue size=4>第三部分:作业提交</font>**

一、实验课下课前提交完成代码，如果下课前未完成，请将已经完成的部分进行提交，未完成的部分于之后的实验报告中进行补充  
要求:  
1)文件格式为：学号-姓名.ipynb  
2)【不要】提交文件夹、压缩包、数据集等无关文件，只需提交单个ipynb文件即可，如果交错请到讲台前联系助教，删掉之前的错误版本后再进行提交

二、本周不需要提交实验报告，这周的实验课内容和下周的实验课内容汇总到同一个实验报告中，于下下周五实验课(3月31号前)上课前提交报告  
要求：  
1)文件格式为：学号-姓名.pdf  
2)【不要】提交文件夹、压缩包、代码文件、数据集等任何与实验报告无关的文件，只需要提交单个pdf文件即可  
3)文件命名时不需要额外添加“实验几”等额外信息，按照格式提交  
4)每周的实验报告提交地址会变化，且有时间限制，提交时间为下周的实验课开始时，请注意及时提交。

实验四(决策树)的实验报告上交地址:https://workspace.jianguoyun.com/inbox/collect/64eccd2dd9584125be510bb68435b0c6/submit

三、课堂课件获取地址:https://www.jianguoyun.com/p/DQlpUFYQp5WhChiS_q0E  
实验内容获取地址:https://www.jianguoyun.com/p/DbKbP-AQp5WhChi1sa0E